# 1. Preliminaries

## 1.1 Setup Discord
<p>
  <ul>
    <li>Create Discord account, confirm email, set developer mode on in settings</li>
    <li>Create a server</li>
    <li>Go to discord.com/developers/ and create an application</li>
    <li>Create a bot, declare intents, add to server</li>
    
  </ul>
</p>

## 1.2 Install Dependencies

In [ ]:
!pip install discord.py
!pip install colab-env --upgrade

## 1.3 Setup Google Drive Access

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 1.4 Imports

In [ ]:
import os, re
import asyncio
import discord
import colab_env

# 1.5 Setup Environment Variables (Discord Token)

In [ ]:
!cat gdrive/My\ Drive/vars.env

COLAB_ENV = Active
TOKEN = ODk3MjE2NDA3MzI5MjAyMTc3.YWSb5A.W6J4k6CoU9mdDMCGA4xYZyxCkQo


In [ ]:
# Add Discord Bot token to the environment variables
colab_env.envvar_handler.add_env("TEST", "HELLO WORLD", overwrite=True)
colab_env.envvar_handler.add_env("TOKEN", "ODk3MjE2NDA3MzI5MjAyMTc3.YWSb5A.W6J4k6CoU9mdDMCGA4xYZyxCkQo", overwrite=True)
!cat gdrive/My\ Drive/vars.env

COLAB_ENV = Active
TOKEN = ODk3MjE2NDA3MzI5MjAyMTc3.YWSb5A.W6J4k6CoU9mdDMCGA4xYZyxCkQo
TEST = HELLO WORLD


In [ ]:
# Might need to delete variables? This is how!
colab_env.envvar_handler.del_env("TEST")
!cat gdrive/My\ Drive/vars.env

COLAB_ENV = Active
TOKEN = ODk3MjE2NDA3MzI5MjAyMTc3.YWSb5A.W6J4k6CoU9mdDMCGA4xYZyxCkQo


In [ ]:
# To actually get the value stored for a variable
# os.getenv("TOKEN")

# 5. Let's get our bot running!

In [ ]:
# Get our token
TOKEN = os.getenv("TOKEN")

# Declare intents
intents = discord.Intents.all()
intents.members = True

# map of users to their strikes
map = {}

# Instantiate the bot
client = discord.Client(intents=intents)

In [ ]:
# Let's define some bad words for our moderator to detect
bad_words = ['pickle', 'tomato', 'oracle']
bad_words = words_to_expressions(bad_words)
bad_words

From algorithm2: p.*i.*c.*k.*l.*e
From algorithm2: t.*o.*m.*a.*t.*o
From algorithm2: o.*r.*a.*c.*l.*e


['p.*i.*c.*k.*l.*e', 't.*o.*m.*a.*t.*o', 'o.*r.*a.*c.*l.*e']

In [ ]:
# Define our first event
@client.event
async def on_ready():
  general = client.get_channel(897217041210155071)
  await general.send(f'{client.user} has connected to Discord!')

# Listen for messages
@client.event
async def on_message(message):
  # We don't want to call this on our own messages (infinite feedback loop == bad)
  if message.author == client.user:
    return

  # Let's moderate!
  found = [re.search(word, message.content.lower()) for word in bad_words]
  if any(found):
    # Keep track of the offending user's strikes
    if message.author in map.keys():
      map[message.author] += 1
    else:
      map[message.author] = 1
    # If they exceed 3 strikes, that's a ban/kick or whatever it may be
    if map[message.author] >= 3:
      await message.channel.send(f'Sorry {message.author}, you\'re outta here.')
    else:
      await message.channel.send(f"""
        {message.author}, please refrain from such language! Strike: {map[message.author]} ( 3 strikes == Ban )
      """)
  # # Send message to channel, to check that we can read messages that are sent
  # await message.channel.send("Cool, I can read the chat!")

In [ ]:
# client.run(TOKEN)

# Note: In Colab or a Jupyter Notebook, regular client.run() 
# does not work due to conflict with the Notebook event loop and thus
# need to use asyncio to create a COROUTINE on a separate event loop
# (client.run internally calls start, and creates the loop whereas here we need to directly call start)
task = asyncio.get_event_loop().create_task(client.start(TOKEN)) 

In [ ]:
# Stop the task (stop bot)
task.cancel()

True

# 6. Text parsing with regular expressions

In [ ]:
# Define a regular expression for pattern matching
words = ["pickle", "orange"] # some bad words
bad_word = words[0]
text = "i think pickles are good?" # some text to parse
found = re.search(bad_word, text)
found.group(0)

'pickle'

In [ ]:
text = "i think p i c k l e s are good?"
found = re.search(bad_word, text)
if found:
  print(found.group(0))
else:
  print("seems like we didn't find any of the bad words, hmmmm")

seems like we didn't find any of the bad words, hmmmm


In [ ]:
text = "i think p_i_c_k_l_e_s are good?"
"""
'.'  : any character
'\s' : whitespace characters
'\d' : digits
'\w' : unicode character
'*'  : match the previous character zero or more times
'+'  : match the previous character one or more times
'()' : grouping
'[]' : match any one of the characters enclosed
"""
pattern = r".*p.*i.*c.*k.*l.*e.*s.*"
found = re.search(pattern, text)
found.group(0)

'i think p_i_c_k_l_e_s are good?'

In [ ]:
# Algorithm1:
# 1. Create an empty character array
# Loop:
#   2. Append the letter
#   3. Append the .*
# 4. join the character array as a single string with no separator
def words_to_expresions_ver_1(words):
  expressions = []
  for word in words:
    char_list = []
    for letter in word:
      char_list.append(letter)
      char_list.append('.')
      char_list.append('*')
    expression = "".join(char_list)
    print(f"From algorithm1: {expression}")
  return expressions
# Algorithm2:
# 1. Split the word into it's character array -> [letter for letter in word]
# 2. Join character array as string, separate the characters by .*
def words_to_expressions(words):
  expressions = []
  for word in words:
    pattern = ".*".join([letter for letter in word])
    print(f"From algorithm2: {pattern}")
    expressions.append(pattern)
  return expressions
words_to_expresions_ver_1(words)
print()
expressions = words_to_expressions(words)

From algorithm1: p.*i.*c.*k.*l.*e.*
From algorithm1: o.*r.*a.*n.*g.*e.*

From algorithm2: p.*i.*c.*k.*l.*e
From algorithm2: o.*r.*a.*n.*g.*e


In [ ]:
expressions

['p.*i.*c.*k.*l.*e', 'o.*r.*a.*n.*g.*e']

In [ ]:
# We have our text
text = "i think oranges are good?"
# call re.search on every expression in expressions and store as list
found = [re.search(expr, text) for expr in expressions]
print(found)

[None, <re.Match object; span=(8, 19), match='oranges are'>]


In [ ]:
# any(iterable) -> returns True if any of the elements are True (None == False, 0 == False, False == False)
if any(found):
  print("We found a matching expression in the text")
else:
  print("The text is clean!")

We found a matching expression in the text
